In [513]:
import glob
import os
import pandas as pd

In [ ]:
###
### Convert OPP data in CSV format given to us by DFO/MEDS into the SWOB format
###

In [514]:
column_mapping={
#   "Region": "",
  "Name": "stn_nam",
  "WMOID": "wmo_synop_id",
  "MSCID": "msc_id",
#   "Buoy Type": "",
  "Time Stamp": "sampling_time",
  "Latitude": "lat",
  "Longitude": "long",
  "Air Temperature": "avg_air_temp_pst10mts",
  "Air Temperature QA Tag": "avg_air_temp_pst10mts_qa_summary",
  "Water Temperature": "avg_sea_sfc_temp_pst10mts",
  "Water Temperature QA Tag": "avg_sea_sfc_temp_pst10mts_qa_summary",
  "Official Pressure": "avg_stn_pres_pst10mts",
  "Official Pressure QA Tag": "avg_stn_pres_pst10mts_qa_summary",
  "Pressure 1": "avg_stn_pres_pst10mts_1",
  "Pressure 1 QA Tag": "avg_stn_pres_pst10mts_1_qa_summary",
  "Pressure 2": "avg_stn_pres_pst10mts_2",
  "Pressure 2 QA Tag": "avg_stn_pres_pst10mts_2_qa_summary",
  "Wave Height": "avg_wave_hgt_pst20mts",
  "Wave Height QA Tag": "avg_wave_hgt_pst20mts_qa_summary",
  "Peak Wave": "pk_wave_hgt_pst20mts",
  "Peak Wave QA Tag": "pk_wave_hgt_pst20mts_qa_summary",
  "Wave Period": "avg_wave_pd_pst20mts",
  "Wave Period QA Tag": "avg_wave_pd_pst20mts_qa_summary",
  "Offical Wind Direction": "avg_wnd_dir_pst10mts",
  "Offical Wind Direction QA Tag": "avg_wnd_dir_pst10mts_qa_summary",
  "Wind Direction 1": "avg_wnd_dir_pst10mts_1",
  "Wind Direction 1 QA Tag": "avg_wnd_dir_pst10mts_1_qa_summary",
  "Wind Direction 2": "avg_wnd_dir_pst10mts_2",
  "Wind Direction 2 QA Tag": "avg_wnd_dir_pst10mts_2_qa_summary",
  "Official Wind Speed": "avg_wnd_spd_pst10mts",
  "Official Wind Speed QA Tag": "avg_wnd_spd_pst10mts_qa_summary",
  "Wind Speed 1": "avg_wnd_spd_pst10mts_1",
  "Wind Speed 1 QA Tag": "avg_wnd_spd_pst10mts_1_qa_summary",
  "Wind Speed 2": "avg_wnd_spd_pst10mts_2",
  "Wind Speed 2 QA Tag": "avg_wnd_spd_pst10mts_2_qa_summary",
  "Official Wind Gust": "max_avg_wnd_spd_pst10mts",
  "Official Wind Gust QA Tag": "max_avg_wnd_spd_pst10mts_qa_summary",
  "Wind Gust 1": "max_avg_wnd_spd_pst10mts_1",
  "Wind Gust 1 QA Tag": "max_avg_wnd_spd_pst10mts_1_qa_summary",
  "Wind Gust 2": "max_avg_wnd_spd_pst10mts_2",
  "Wind Gust 2 QA Tag": "max_avg_wnd_spd_pst10mts_2_qa_summary",
# These arent in the SWOB
  "Region": "region",
  "Buoy Type": "buoy_type",
  "Compass 1": "compass_1",
  "Compass 1 QA Tag": "compass_1_qa_summary",
  "Compass 2": "compass_2",
  "Compass 2 QA Tag": "compass_2_qa_summary"
}


In [516]:
# Load all OPP CSV data
opp_csv_paths = glob.glob("O*.csv")

opp_csv_data=pd.DataFrame
csv_datas=[]
for csv_path in opp_csv_paths:
    df_csv=pd.read_csv(csv_path,na_values=['MSNG'])
    csv_datas.append(df_csv)
df=pd.concat(csv_datas).drop_duplicates()
    

In [517]:
# The CSVs have columns like "Temperature,QA,Wind speed,QA"
# This adds the variable name before "QA" so the column
# names are unique
new_column_names=[]
for column in df.columns:
    if "QA Tag" in column:
        new_column_names.append(previous_col + " QA Tag")
    else:
        new_column_names.append(column)
        previous_col=column
df.columns=new_column_names

In [518]:
#
# Reformat the data a bit to match SWOB
#

In [519]:
# remove the '00' from WMOID to match format used in SWOB system. Eg 4600303 to 46303
df.WMOID = (df.WMOID.astype(str).str.slice(0,2)+ df.WMOID.astype(str).str.slice(4,))

In [521]:
# Convert wind speeds from m/s  to km/h to match SWOB
wind_speed_columns=["Official Wind Speed",
                    "Wind Speed 1",
                    "Wind Speed 2",
                    "Official Wind Gust",
                    "Wind Gust 1",
                    "Wind Gust 2"
                   ]

for column in wind_speed_columns:
    df[column]=(df[column]*3.6).round(decimals=4)


In [522]:
# Change CSV column names to SWOB column names
df.rename(columns=column_mapping,inplace=True)

In [523]:
# remove CSV columns that havent mapped to SWOB columns
swob_columns = column_mapping.values()
for column in df.columns:
    if column not in swob_columns:
        print("Column not in database table:",column)


In [524]:
# Keep track of the source of the data in the DB (SWOB vs CSV)
df.source="CSV"

In [525]:
# string to datetime (UTC)
df['sampling_time'] = pd.to_datetime(df['sampling_time'])

In [526]:
df.to_csv("opp_reformatted2.csv",index=False)